In [1]:
from nipype.interfaces.fsl import preprocess
from nipype.interfaces.fsl import ImageMaths
from nilearn import plotting
%matplotlib inline
from niwidgets import NiftiWidget
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from IPython.display import Javascript
import nilearn.plotting as nip#for using nilearn plot functions
from nilearn.image.image import mean_img,load_img

191014-16:08:05,540 nipype.utils INFO:
	 Running nipype version 1.3.0-dev (latest: 1.2.3)


In [2]:
results_path = "../../results/output/preproc/*"

In [3]:
list_subjs = !ls -d $results_path | grep session

In [4]:
def run_cells_below():
    display(Javascript("Jupyter.notebook.execute_cell_range(6,10)"))

In [5]:
subj_id_selector = widgets.Select(
    options=list_subjs,
    # value='C575',
    # rows=10,
    description='Subject Id:',
    disabled=False
)
#subj_id_selector.on_trait_change(run_cells_below)
display(subj_id_selector)

Select(description='Subject Id:', options=('../../results/output/preproc/_session_id_1_subject_id_C113', '../.…

In [6]:
mni_template = "../../datos/TEMPLATES/MNI152_T1_1mm.nii.gz"
mni_3mm_template = "../../datos/TEMPLATES/MNI152_T1_3mm.nii.gz"
cbl_template = "../../datos/TEMPLATES/Cerebellum-MNIflirt-maxprob-thr0-1mm.nii.gz"
neck_filename = subj_id_selector.value+"/Neck_remover/mprage_ROI.nii.gz"
skull_filename = subj_id_selector.value+"/skull_strip/mprage_ROI_flirt_brain.nii.gz"
bold_filename = subj_id_selector.value+"/applyTransFunc/f1_trim_st_mcf_trans.nii.gz"
print(subj_id_selector.value)

../../results/output/preproc/_session_id_1_subject_id_C503


In [7]:
maths = ImageMaths(in_file=cbl_template, op_string= '-bin')
maths.verbose = True
bin_result = maths.run()

In [8]:
anat_widget = NiftiWidget(bin_result.outputs.out_file)
anat_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=90, continuous_update=False, description='x', max=181), IntSlider(value=…

In [9]:
cbl_img = load_img(bin_result.outputs.out_file)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 182 218 182   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  1.  1.  1.  1.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'6.0.1'
aux_file        : b'Random-Rainbow'
qform_code      : mni
sform_code      : mni
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [-1.  0.  

In [19]:
print(cbl_img.header["dim"])

[  3 182 218 182   1   1   1   1]


In [10]:
bold_img = load_img(bold_filename)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  4 182 218 182 158   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [-1.    1.    1.    1.    1.82  0.    0.    0.  ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : -0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [0. 0

In [18]:
print(bold_img.header["dim"])

[  4 182 218 182 158   1   1   1]


In [11]:
masking = ImageMaths()
masking.inputs.in_file = bold_filename
masking.inputs.mask_file = bin_result.outputs.out_file
masking.verbose = True
masking_results = masking.run()

In [12]:
mean_cbl = mean_img(masking_results.outputs.out_file)

In [16]:
masking_results.outputs.out_file
!cp /home/elekin/pyrestfmri/notebooks/f1_trim_st_mcf_trans_maths.nii.gz /home/elekin/results/

In [14]:
mean_cbl.get_data().shape

(182, 218, 182)

In [13]:
roi_widget = NiftiWidget(mean_cbl)
roi_widget.nifti_plotter(plotting_func=nip.plot_roi, bg_img=mni_template)

<Figure size 432x288 with 0 Axes>

interactive(children=(Text(value='../../datos/TEMPLATES/MNI152_T1_1mm.nii.gz', description='bg_img'), Dropdown…